In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

In [ ]:
%load_ext nb_mypy

# How to Compute the Conjunctive Normal Form

Formulas are represented as nested tuples.  In order to convert a string into a nested tuple we use the *parser* that is found in the notebook `Propositional-Logic-Parser.ipynb`.

In [ ]:
%%capture
%run Propositional-Logic-Parser.ipynb

In [ ]:
from typing import TypeVar

In [ ]:
Variable = str
Formula  = TypeVar('Formula')
Formula  = Variable | tuple[Formula, ...]
Literal  = Variable | tuple[str, Variable]
Clause   = frozenset[Literal]
CNF      = set[Clause]

In [ ]:
def parse(s: str) -> Formula:
    parser = LogicParser(s) # type: ignore
    return parser.parse()   # type ignore

The function `eliminateBiconditional(f)` takes a formula `f` from propositional logic and eliminates all occurrences of the operator '↔' from this formula.  This is done by using the following equivalence:
$$ (g \leftrightarrow h) \;\Leftrightarrow\; (g \rightarrow h) \wedge (h \rightarrow g) $$

In [ ]:
def eliminateBiconditional(f: Formula) -> Formula:
    'Eliminate the logical operator "↔" from the formula f.'
    match f:
        case str(p):        # This case covers variables.
            return p
        case ('↔', g, h):
            return eliminateBiconditional( ('∧', ('→', g, h), ('→', h, g)) )
        case ('⊤', ) | ('⊥', ):
            return f
        case ('¬', g):
            return ('¬', eliminateBiconditional(g))
        case (op, g, h):   # This case covers '→', '∧', and '∨'.
            return (op, eliminateBiconditional(g), eliminateBiconditional(h))
    return None # type: ignore

The function $\texttt{eliminateConditional}(f)$ takes a formula $f$ from propositional logic and eliminates all occurrences of the operator '→' from this formula.  This is done by using the following equivalence:
$$ (g \rightarrow h) \;\Leftrightarrow\; (\neg g \vee h) $$

In [ ]:
def eliminateConditional(f: Formula) -> Formula:
    'Eliminate the logical operator "→" from f.'
    match f:
        case str(p):    # variables
            return p
        case ('⊤', ) | ('⊥', ):
            return f
        case ('→', g, h): 
            return eliminateConditional(('∨', ('¬', g), h))
        case ('¬', g):
            return ('¬', eliminateConditional(g))
        case (op, g, h):      # This case covers '∧' and '∨'.
            return (op, eliminateConditional(g), eliminateConditional(h))
    return None # type: ignore

The function $\texttt{nnf}(f)$ computes the *negation normal form* of $f$, while $\texttt{neg}(f)$ computes the *negation normal form* of $\neg f$.  The expression $\texttt{nnf}(f)$ is defined recursively as follows:
<ol>
    <li> $\texttt{nnf}(\neg \texttt{F}) = \texttt{neg}(\texttt{F})$, </li>
    <li> $\texttt{nnf}(\texttt{F}_1 \wedge \texttt{F}_2) = 
          \texttt{nnf}(\texttt{F}_1) \wedge \texttt{nnf}(\texttt{F}_2)$,</li>
    <li> $\texttt{nnf}(\texttt{F}_1 \vee \texttt{F}_2) = 
          \texttt{nnf}(\texttt{F}_1) \vee \texttt{nnf}(\texttt{F}_2)$.</li>
</ol>
The auxiliary function $\texttt{neg}$ is also defined recursively:
<ol>
    <li> $\texttt{neg}(p) = \texttt{nnf}(\neg p) = \neg p$ for all propositional variables $p$,</li>
    <li> $\texttt{neg}(\neg F) = \texttt{nnf}(\neg \neg F) = \texttt{nnf}(F)$,</li>
    <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(F_1 \wedge F_2 \bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg(F_1 \wedge F_2)\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1 \vee \neg F_2\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1\bigr) \vee \texttt{nnf}\bigl(\neg F_2\bigr) \\[0.1cm]
       = & \texttt{neg}(F_1) \vee \texttt{neg}(F_2).
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(F_1 \wedge F_2 \bigr) = \texttt{neg}(F_1) \vee \texttt{neg}(F_2)$.</li>
     <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(F_1 \vee F_2 \bigr)        \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg(F_1 \vee F_2) \bigr)  \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1 \wedge \neg F_2 \bigr)  \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1\bigr) \wedge \texttt{nnf}\bigl(\neg F_2 \bigr)  \\[0.1cm]
       = & \texttt{neg}(F_1) \wedge \texttt{neg}(F_2). 
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(F_1 \vee F_2 \bigr) = \texttt{neg}(F_1) \wedge \texttt{neg}(F_2)$.</li>
</ol>

The forward declaration for the function `neg` is needed to typecheck the function `nnf`.

In [ ]:
def neg(f: Formula) -> Formula:
    return None # type: ignore

In [ ]:
def nnf(f: Formula) -> Formula:
    'Compute the negation normal form of f.'
    match f:
        case str(p): 
            return p
        case ('⊤', ) | ('⊥', ):
            return f
        case ('¬', g):
            return neg(g)
        case (op, g, h):
            return (op, nnf(g), nnf(h))
    return None # type: ignore

In [ ]:
def neg(f: Formula) -> Formula:
    'Compute the negation normal form of ¬f.'
    match f:
        case str(p): 
            return ('¬', p)
        case ('⊤', ):
            return ('⊥', )
        case ('⊥', ):
            return ('⊤', )
        case ('¬', g):
            return nnf(g)
        case ('∧', g, h):
            return ('∨', neg(g), neg(h))
        case ('∨', g, h):
            return ('∧', neg(g), neg(h))
    return None # type: ignore

The function $\texttt{cnf}(f)$ takes a formula $f$ that is in *negation normal form*, i.e. the negation operator is only applied to propositional variables and returns the *conjunctive normal form* of $f$ in *set notation*.  In order to achieve
this it uses the distributive law
$$ (f \wedge g) \vee (h \wedge k) \Leftrightarrow (f \vee h) \wedge (f \vee k) \wedge (g \vee h) \wedge (g \vee k). $$

In [ ]:
def cnf(f: Formula) -> CNF:
    match f:
        case str(p):         # f is a variable
            return { frozenset({p}) }
        case ('⊤', ):
            return set()
        case ('⊥', ):
            return { frozenset() }
        case ('¬', p):
            return { frozenset({ ('¬', p) }) }  # f is a negative literal
        case ('∧', g, h):
            return cnf(g) | cnf(h)
        case ('∨', g, h):
            return { k1 | k2 for k1 in cnf(g) for k2 in cnf(h) }
    return None # type: ignore

The function $\texttt{isTrivial}(C)$ checks whether the clause $C$ is *trivial*.

In [ ]:
def isTrivial(Clause: Clause) -> bool:
    return any(('¬', p) in Clause for p in Clause)

The function $\texttt{simplify}(Cs)$ takes a set of clauses and removes all trivial clauses from $Cs$.

In [ ]:
def simplify(Clauses: set[Clause]) -> set[Clause]:
    return { C for C in Clauses if not isTrivial(C) }

The function $f$ takes a propositional formula $f$ and transforms $f$ into *conjunctive normal form*.  
Furthermore, trivial clausues are removed.

In [ ]:
def normalize(f: Formula) -> CNF:
    n1 = eliminateBiconditional(f)
    n2 = eliminateConditional(n1)
    n3 = nnf(n2)
    n4 = cnf(n3)
    return simplify(n4)

In [ ]:
def prettify[E](M: set[frozenset[E]]) -> str:
    """Turn the set of frozen sets M into a string that looks like a set of sets.
       M is assumed to be the power set of some set.
    """
    if M == set():
        return '{}'
    result = "{"
    for A in M:
        if A == frozenset(): 
            result += "{}, "
        else:
            result += str(set(A)) + ", " # A is converted from a frozen set to a set
    result = result[:-2] # remove the trailing substring ", "
    result += "}"
    return result

In [ ]:
def test(s: str) -> str:
    f = parse(s)
    print(f'The knf of {s} is:')
    return prettify(normalize(f))

In [ ]:
test('(¬p → q) → (p → q) → q')

In [ ]:
test('(a → b) ↔ (¬a ∧ ¬b)')

In [ ]:
test('(p ∧ q → r) ∨ ¬r → ¬p')

In [ ]:
test('⊤')

In [ ]:
test('⊥')

In [ ]:
test('(p ∧ q → r) ∨ ¬r → ¬p ↔ ¬p')